In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3150/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_1_9_24,0.956065,0.940697,0.698046,0.817152,0.048623,0.023170,0.144304,0.080175,"Hidden Size=[25, 10], regularizer=0.02, learni..."
1,model_1_9_23,0.955813,0.940895,0.708432,0.822575,0.048902,0.023093,0.139340,0.077797,"Hidden Size=[25, 10], regularizer=0.02, learni..."
2,model_1_9_22,0.955519,0.940563,0.719456,0.828072,0.049228,0.023223,0.134072,0.075387,"Hidden Size=[25, 10], regularizer=0.02, learni..."
3,model_1_9_21,0.955158,0.940379,0.729831,0.833307,0.049627,0.023295,0.129114,0.073092,"Hidden Size=[25, 10], regularizer=0.02, learni..."
4,model_1_9_20,0.954744,0.939602,0.740620,0.838472,0.050085,0.023598,0.123958,0.070827,"Hidden Size=[25, 10], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...
336,model_11_5_14,0.873971,0.688888,0.795026,0.785309,0.139477,0.036576,0.218466,0.122170,"Hidden Size=[45, 20], regularizer=0.1, learnin..."
339,model_11_7_1,0.873659,0.575082,0.863268,0.805765,0.139823,0.067749,0.098368,0.082158,"Hidden Size=[45, 20], regularizer=0.1, learnin..."
344,model_9_4_5,0.872826,0.930945,0.737530,0.856101,0.140744,0.141083,0.389578,0.258022,"Hidden Size=[45, 20], regularizer=0.02, learni..."
345,model_1_6_4,0.872735,0.909020,0.918195,0.912447,0.140845,0.150972,0.090288,0.122415,"Hidden Size=[25, 10], regularizer=0.02, learni..."
